In [76]:
import pandas as pd
import pathlib
from pathlib import Path
from sklearn import svm
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.svm import SVR
from sklearn.linear_model import Ridge
import numpy as np
import xgboost as xgb
from tqdm import tqdm
import random

pd.set_option('display.max_columns', None)

# Import data

In [291]:
data = pd.read_csv(str(Path.cwd()) + '/final_updated.csv')
data = data.loc[data['OffPoss'] != 0]
home = data[data.columns[data.columns.isin([col for col in data.columns if col[:2] != "XX"])]]
home = home.drop(["Date", "GameID", "TeamID", "Season", "Team"], axis=1)
away = data[data.columns[data.columns.isin([col for col in data.columns if col[:2] == "XX"])]]
away = away.drop(["XXTeamID", "XXTeam"], axis=1)

In [42]:
cols = list(home.columns)
home_rows = []
for i in tqdm(range(len(home))):
    row = {str(col): home.iloc[i, :][col] - away.iloc[i, :]["XX" + col] for col in cols if col != "HomeWinPct"}
    row["HomeWinPct"] = home.iloc[i, :]["HomeWinPct"]
    home_rows.append(row)

100%|█████████████████████████████████████| 10610/10610 [04:50<00:00, 36.52it/s]


In [45]:
cols = list(away.columns)
cols.remove("XXAwayWinPct")
away_rows = []
for i in tqdm(range(len(away))):
    row = {str(col): away.iloc[i, :][col] - home.iloc[i, :][col[2:]] for col in cols if col != "XXAwayWinPct"}
    row["XXAwayWinPct"] = away.iloc[i, :]["XXAwayWinPct"]
    away_rows.append(row)

100%|█████████████████████████████████████| 10610/10610 [04:48<00:00, 36.82it/s]


# Model Testing

In [112]:
data = pd.read_csv(str(Path.cwd()) + '/final_updated.csv')
data = data.loc[data['OffPoss'] != 0]
data = data.dropna()
X = data[["Elo", "XXElo", "Points", "XXPoints", "Wins", "XXWins", "OffRank",
          "DefRank", "XXOffRank", "XXDefRank"]]
y = data["XXTemp"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=21, shuffle=True)

In [78]:
tests = []
for solver in tqdm(["auto", "svd", "cholesky", "lsqr", "sparse_cg", "sag", "saga", "lbfgs"]):
    for alpha in [1 + i for i in range(10)]:
        model = Ridge(alpha=alpha, solver=solver)
        scores = []
        print('A: ' + str(alpha) + '| Solver: ' + solver)
        print('-----------------------------')
        model.fit(X_train.values, y_train.values)
        y_pred = svr.predict(X_test.values)
        score = metrics.mean_squared_error(y_test, y_pred)
        print(score)
        tests.append({"A": alpha, "Solver": solver, "Score": score})

  0%|                                                     | 0/8 [00:00<?, ?it/s]

A: 1| Solver: auto
-----------------------------
125.66705018717947
A: 2| Solver: auto
-----------------------------
125.66705018717947
A: 3| Solver: auto
-----------------------------
125.66705018717947
A: 4| Solver: auto
-----------------------------
125.66705018717947
A: 5| Solver: auto
-----------------------------
125.66705018717947
A: 6| Solver: auto
-----------------------------
125.66705018717947
A: 7| Solver: auto
-----------------------------
125.66705018717947
A: 8| Solver: auto
-----------------------------
125.66705018717947
A: 9| Solver: auto
-----------------------------
125.66705018717947
A: 10| Solver: auto
-----------------------------


 12%|█████▋                                       | 1/8 [00:18<02:09, 18.50s/it]

125.66705018717947
A: 1| Solver: svd
-----------------------------
125.66705018717947
A: 2| Solver: svd
-----------------------------
125.66705018717947
A: 3| Solver: svd
-----------------------------
125.66705018717947
A: 4| Solver: svd
-----------------------------
125.66705018717947
A: 5| Solver: svd
-----------------------------
125.66705018717947
A: 6| Solver: svd
-----------------------------
125.66705018717947
A: 7| Solver: svd
-----------------------------
125.66705018717947
A: 8| Solver: svd
-----------------------------
125.66705018717947
A: 9| Solver: svd
-----------------------------
125.66705018717947
A: 10| Solver: svd
-----------------------------


 25%|███████████▎                                 | 2/8 [00:37<01:52, 18.75s/it]

125.66705018717947
A: 1| Solver: cholesky
-----------------------------
125.66705018717947
A: 2| Solver: cholesky
-----------------------------
125.66705018717947
A: 3| Solver: cholesky
-----------------------------
125.66705018717947
A: 4| Solver: cholesky
-----------------------------
125.66705018717947
A: 5| Solver: cholesky
-----------------------------
125.66705018717947
A: 6| Solver: cholesky
-----------------------------
125.66705018717947
A: 7| Solver: cholesky
-----------------------------
125.66705018717947
A: 8| Solver: cholesky
-----------------------------
125.66705018717947
A: 9| Solver: cholesky
-----------------------------
125.66705018717947
A: 10| Solver: cholesky
-----------------------------


 38%|████████████████▉                            | 3/8 [00:56<01:33, 18.74s/it]

125.66705018717947
A: 1| Solver: lsqr
-----------------------------
125.66705018717947
A: 2| Solver: lsqr
-----------------------------
125.66705018717947
A: 3| Solver: lsqr
-----------------------------
125.66705018717947
A: 4| Solver: lsqr
-----------------------------
125.66705018717947
A: 5| Solver: lsqr
-----------------------------
125.66705018717947
A: 6| Solver: lsqr
-----------------------------
125.66705018717947
A: 7| Solver: lsqr
-----------------------------
125.66705018717947
A: 8| Solver: lsqr
-----------------------------


 38%|████████████████▉                            | 3/8 [01:11<01:58, 23.76s/it]


KeyboardInterrupt: 

In [113]:
test = []
for i in range(5):
    model = Ridge(alpha=1, solver="sag", random_state=i)
    model.fit(X_train.values, y_train.values)
    y_pred = svr.predict(X_test.values)
    score = metrics.mean_squared_error(y_test, y_pred)
    test.append({"Score": score, "Preds": y_pred})

In [48]:
away_tests = []
for kernel in tqdm(['poly', 'rbf', 'linear']):
    if kernel == 'linear':
        c_vals = [2**-10, 2**-8, 2**-6, 2**-5.5, 2**-5, 2**-4.5, 2**-4]
    elif kernel == 'poly':
        c_vals = [100 + (i * 4) for i in range(20)]
    else:
        c_vals = [500 + (i * 25) for i in range(20)]
    for c in c_vals:
        scores = []
        print('C: ' + str(c) + '| Kernel: ' + kernel)
        print('-----------------------------')
        svr = SVR(kernel=kernel, C=c)
        svr.fit(X_train.values, y_train.values)
        y_pred = svr.predict(X_test.values)
        score = metrics.mean_squared_error(y_test, y_pred)
        print(score)
        away_tests.append({"C": c, "Kernel": kernel, "Score": score})

  0%|                                                     | 0/3 [00:00<?, ?it/s]

C: 100| Kernel: poly
-----------------------------
126.89012346699002
C: 104| Kernel: poly
-----------------------------
126.89201137391231
C: 108| Kernel: poly
-----------------------------
126.84714900813903
C: 112| Kernel: poly
-----------------------------
126.822999136304
C: 116| Kernel: poly
-----------------------------
126.80294829154951
C: 120| Kernel: poly
-----------------------------
126.79170571462548
C: 124| Kernel: poly
-----------------------------


  0%|                                                     | 0/3 [00:18<?, ?it/s]


KeyboardInterrupt: 

In [44]:
home_scores = tests

In [59]:
home_scores

[{'C': 20, 'Kernel': 'poly', 'Score': 128.90437951044052},
 {'C': 22, 'Kernel': 'poly', 'Score': 128.27916730312685},
 {'C': 24, 'Kernel': 'poly', 'Score': 127.76515030336272},
 {'C': 26, 'Kernel': 'poly', 'Score': 127.38831891085607},
 {'C': 28, 'Kernel': 'poly', 'Score': 127.04305263980683},
 {'C': 30, 'Kernel': 'poly', 'Score': 126.77429576807768},
 {'C': 32, 'Kernel': 'poly', 'Score': 126.4467071017529},
 {'C': 34, 'Kernel': 'poly', 'Score': 126.18204592610127},
 {'C': 36, 'Kernel': 'poly', 'Score': 125.95645929375345},
 {'C': 38, 'Kernel': 'poly', 'Score': 125.74316667718763},
 {'C': 40, 'Kernel': 'poly', 'Score': 125.60260419974553},
 {'C': 42, 'Kernel': 'poly', 'Score': 125.48235486615263},
 {'C': 44, 'Kernel': 'poly', 'Score': 125.32302346250393},
 {'C': 46, 'Kernel': 'poly', 'Score': 125.21151646507931},
 {'C': 48, 'Kernel': 'poly', 'Score': 125.07239680461201},
 {'C': 50, 'Kernel': 'poly', 'Score': 124.95737139971348},
 {'C': 52, 'Kernel': 'poly', 'Score': 124.88801739766427}

In [21]:
svr = SVR(kernel="rbf", C=2**15)
svr.fit(X_train.values, y_train.values)
y_pred = svr.predict(X_test.values)
score = metrics.mean_squared_error(y_test, y_pred)

In [22]:
score

125.66705018717947

In [4]:
tests = []
for kernel in tqdm(['poly', 'rbf', 'linear']):
    if kernel == 'linear':
        c_vals = [2**-6, 2**-5.5, 2**-5, 2**-4.5, 2**-4, 2**-3, 2**-2, 2**-1, 1, 2, 2**2, 2**3, 2**4, 2**6]
    elif kernel == 'poly':
        c_vals = [2**3, 9, 10, 11, 12, 13, 14, 15, 16]
    else:
        c_vals = [16 + i for i in range(49)] + [2**8, 260, 264, 300]
    for c in c_vals:
        scores = []
        print('C: ' + str(c) + '| Kernel: ' + kernel)
        print('-----------------------------')
        clf = svm.SVC(kernel=kernel, C=c, random_state=42)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        print(score)
        tests.append({"C": c, "Kernel": kernel, "Score": score})

  0%|                                                     | 0/3 [00:00<?, ?it/s]

C: 8| Kernel: poly
-----------------------------
0.6722470238095238
C: 9| Kernel: poly
-----------------------------
0.671875
C: 10| Kernel: poly
-----------------------------
0.6722470238095238
C: 11| Kernel: poly
-----------------------------
0.6748511904761905
C: 12| Kernel: poly
-----------------------------
0.6722470238095238
C: 13| Kernel: poly
-----------------------------
0.6733630952380952
C: 14| Kernel: poly
-----------------------------
0.6729910714285714
C: 15| Kernel: poly
-----------------------------
0.6741071428571429
C: 16| Kernel: poly
-----------------------------


 33%|██████████████▋                             | 1/3 [09:08<18:17, 548.66s/it]

0.6722470238095238
C: 16| Kernel: rbf
-----------------------------
0.6722470238095238
C: 17| Kernel: rbf
-----------------------------
0.671875
C: 18| Kernel: rbf
-----------------------------
0.6726190476190477
C: 19| Kernel: rbf
-----------------------------
0.6737351190476191
C: 20| Kernel: rbf
-----------------------------
0.6729910714285714
C: 21| Kernel: rbf
-----------------------------
0.6729910714285714
C: 22| Kernel: rbf
-----------------------------
0.6729910714285714
C: 23| Kernel: rbf
-----------------------------
0.6729910714285714
C: 24| Kernel: rbf
-----------------------------
0.6729910714285714
C: 25| Kernel: rbf
-----------------------------
0.6726190476190477
C: 26| Kernel: rbf
-----------------------------
0.6733630952380952
C: 27| Kernel: rbf
-----------------------------
0.6733630952380952
C: 28| Kernel: rbf
-----------------------------
0.6729910714285714
C: 29| Kernel: rbf
-----------------------------
0.6733630952380952
C: 30| Kernel: rbf
--------------------

 67%|█████████████████████████████▎              | 2/3 [15:02<07:14, 434.16s/it]

0.6744791666666666
C: 0.015625| Kernel: linear
-----------------------------
0.671875
C: 0.02209708691207961| Kernel: linear
-----------------------------
0.671875
C: 0.03125| Kernel: linear
-----------------------------
0.671875
C: 0.04419417382415922| Kernel: linear
-----------------------------
0.671875
C: 0.0625| Kernel: linear
-----------------------------
0.671875
C: 0.125| Kernel: linear
-----------------------------
0.671875
C: 0.25| Kernel: linear
-----------------------------
0.671875
C: 0.5| Kernel: linear
-----------------------------
0.671875
C: 1| Kernel: linear
-----------------------------
0.671875
C: 2| Kernel: linear
-----------------------------
0.671875
C: 4| Kernel: linear
-----------------------------


 67%|████████████████████████████▋              | 2/3 [34:48<17:24, 1044.38s/it]


KeyboardInterrupt: 

In [11]:
tests = sorted(tests, key=lambda x: x["Score"], reverse=True)

In [13]:
tests

[{'C': 46, 'Kernel': 'rbf', 'Score': 0.6752232142857143},
 {'C': 11, 'Kernel': 'poly', 'Score': 0.6748511904761905},
 {'C': 45, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 47, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 54, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 55, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 56, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 57, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 58, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 59, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 60, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 61, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 62, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 256, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 260, 'Kernel': 'rbf', 'Score': 0.6748511904761905},
 {'C': 43, 'Kernel': 'rbf', 'Score': 0.6744791666666666},
 {'C': 44, 'Kernel': 'rbf', 'Score': 0.6744791666666666},
 {'C': 48, 

In [ ]:
clf = svm.SVC(kernel="rbf", C=46, random_state=42)

# Getting data for simulations

In [23]:
def find_start(df):
    seen = []
    for i, row in df.iterrows():
        if len(seen) == 30:
            start = row["Date"]
            break
            
        if row["Team"] not in seen:
            seen.append(row["Team"])

        if row["XXTeam"] not in seen:
            seen.append(row["XXTeam"])

    for idx, row in df[i:].iterrows():
        if row["Date"] != start:
            return idx
            
    return i

In [24]:
def get_recent(pre, team):
    i = len(pre) - 1
    while(i != 0):
        if pre.iloc[i]["Team"] == team or pre.iloc[i]["XXTeam"] == team:
            return pre.iloc[i]
        
        i -= 1
        
    raise ValueError()

In [25]:
def new_elo(row):
    homePoints = row["Temp"]
    homeElo = row["Elo"]
    awayPoints = row["XXTemp"]
    awayElo = row["XXElo"]
    transformHome = 10**(homeElo/400)
    transformAway = 10**(awayElo/400)
    expectHome = transformHome/(transformHome+transformAway)
    expectAway = transformAway/(transformHome+transformAway)
    eloDif = (homeElo+100)-awayElo
    
    if homePoints - awayPoints > 0:
        movm = ((homePoints-awayPoints+3)**0.8)/(7.5+(0.006*eloDif))
        eloChange = int((20*(1-expectHome))*movm)
        newHome = homeElo + eloChange
        newAway = awayElo - eloChange
    else:
        movm = ((awayPoints-homePoints+3)**0.8)/(7.5+(0.006*eloDif))
        eloChange = int((20*(1-expectAway))*movm)
        newAway = awayElo + eloChange
        newHome = homeElo - eloChange
    
    return {row["Team"]: newHome, row["XXTeam"]: newAway}

In [48]:
# This function gives a ranking for a team's offense and defense when they are home and when they are away.

def stat_ranking(df, index, stat_type, off_def, home_away):
    column = list(df.columns).index(home_away)
    df.reset_index(inplace=True, drop=True)
    team = df.iloc[index, column]
    previous_rows = df.iloc[:index]
    
    if off_def == 'off':
        reverse_val = True
    elif off_def == 'def':
        reverse_val = False
        
    averages = sorted([(item[stat_type].iloc[-1], item[home_away].iloc[0]) for key, item in previous_rows.groupby([home_away])],
                      key=lambda x: x[0], reverse=reverse_val)
    try:
        return int([x[1] for x in averages].index(team) + 1)
    except:
        return None

In [27]:
def get_gp(df, team, game_id):
    ix = df.loc[df["GameID"] == game_id].index[0]
    prev = df.iloc[:ix]
    return len(prev.loc[(prev["Team"] == team) | (prev["XXTeam"] == team)])

In [28]:
def compute_avg(recent, team, gp):
    num = (recent["Points"] * gp) + recent["Temp"]
    if recent["XXTeam"] == team:
        num = (recent["XXPoints"] * gp) + recent["XXTemp"]
        
    den = (gp + 1)
    return num / den

In [30]:
def get_wins(recent, team):
    wins = recent["Wins"]
    if recent["XXTeam"] == team:
        wins = recent["XXWins"]
        if recent["XXTemp"] > recent["Temp"]:
            wins += 1
    elif recent["Temp"] > recent["XXTemp"]:
        wins += 1
        
    return wins

In [271]:
data[0]

,Date,GameID,H/A,Team,ML,Wins,WinLast10,Elo,EloLast10,OffPoss,Points,FG2M,FG2A,Fg2Pct,FG3M,FG3A,Fg3Pct,NonHeaveFg3Pct,FtPoints,PtsAssisted2s,PtsUnassisted2s,PtsAssisted3s,PtsUnassisted3s,Assisted2sPct,NonPutbacksAssisted2sPct,Assisted3sPct,FG3APct,ShotQualityAvg,EfgPct,TsPct,PtsPutbacks,Fg2aBlocked,FG2APctBlocked,Fg3aBlocked,FG3APctBlocked,AtRimFG3AFrequency,Avg2ptShotDistance,Avg3ptShotDistance,AtRimFGM,AtRimFGA,AtRimFrequency,AtRimAccuracy,UnblockedAtRimAccuracy,AtRimPctAssisted,AtRimPctBlocked,ShortMidRangeFGM,ShortMidRangeFGA,ShortMidRangeFrequency,ShortMidRangeAccuracy,UnblockedShortMidRangeAccuracy,ShortMidRangePctAssisted,ShortMidRangePctBlocked,LongMidRangeFGM,LongMidRangeFGA,LongMidRangeFrequency,LongMidRangeAccuracy,UnblockedLongMidRangeAccuracy,LongMidRangePctAssisted,LongMidRangePctBlocked,Corner3FGM,Corner3FGA,Corner3Frequency,Corner3Accuracy,UnblockedCorner3Accuracy,Corner3PctAssisted,Corner3PctBlocked,Arc3FGM,Arc3FGA,Arc3Frequency,Arc3Accuracy,UnblockedArc3Accuracy,Arc3PctAssisted,Arc3PctBlocked,NonHeaveArc3FGM,NonHeaveArc3FGA,NonHeaveArc3Accuracy,HeaveAttempts,HeaveMakes,Assists,AssistPoints,TwoPtAssists,ThreePtAssists,AtRimAssists,ShortMidRangeAssists,LongMidRangeAssists,Corner3Assists,Arc3Assists,Rebounds,DefRebounds,FTDefRebounds,DefFTReboundPct,DefTwoPtRebounds,DefTwoPtReboundPct,DefThreePtRebounds,DefThreePtReboundPct,DefFGReboundPct,OffRebounds,FTOffRebounds,OffFTReboundPct,OffTwoPtRebounds,OffTwoPtReboundPct,OffThreePtRebounds,OffThreePtReboundPct,OffFGReboundPct,DefAtRimReboundPct,DefShortMidRangeReboundPct,DefLongMidRangeReboundPct,DefArc3ReboundPct,DefCorner3ReboundPct,OffAtRimReboundPct,OffShortMidRangeReboundPct,OffLongMidRangeReboundPct,OffArc3ReboundPct,OffCorner3ReboundPct,SelfOReb,SelfORebPct,Turnovers,LiveBallTurnovers,DeadBallTurnovers,LiveBallTurnoverPct,LostBallTurnovers,LostBallOutOfBoundsTurnovers,BadPassTurnovers,BadPassOutOfBoundsTurnovers,Travels,3SecondViolations,StepOutOfBoundsTurnovers,OffensiveGoaltends,XXH/A,XXTeam,XXML,XXWins,XXWinLast10,XXElo,XXEloLast10,XXOffPoss,XXPoints,XXFG2M,XXFG2A,XXFg2Pct,XXFG3M,XXFG3A,XXFg3Pct,XXNonHeaveFg3Pct,XXFtPoints,XXPtsAssisted2s,XXPtsUnassisted2s,XXPtsAssisted3s,XXPtsUnassisted3s,XXAssisted2sPct,XXNonPutbacksAssisted2sPct,XXAssisted3sPct,XXFG3APct,XXShotQualityAvg,XXEfgPct,XXTsPct,XXPtsPutbacks,XXFg2aBlocked,XXFG2APctBlocked,XXFg3aBlocked,XXFG3APctBlocked,XXAtRimFG3AFrequency,XXAvg2ptShotDistance,XXAvg3ptShotDistance,XXAtRimFGM,XXAtRimFGA,XXAtRimFrequency,XXAtRimAccuracy,XXUnblockedAtRimAccuracy,XXAtRimPctAssisted,XXAtRimPctBlocked,XXShortMidRangeFGM,XXShortMidRangeFGA,XXShortMidRangeFrequency,XXShortMidRangeAccuracy,XXUnblockedShortMidRangeAccuracy,XXShortMidRangePctAssisted,XXShortMidRangePctBlocked,XXLongMidRangeFGM,XXLongMidRangeFGA,XXLongMidRangeFrequency,XXLongMidRangeAccuracy,XXUnblockedLongMidRangeAccuracy,XXLongMidRangePctAssisted,XXLongMidRangePctBlocked,XXCorner3FGM,XXCorner3FGA,XXCorner3Frequency,XXCorner3Accuracy,XXUnblockedCorner3Accuracy,XXCorner3PctAssisted,XXCorner3PctBlocked,XXArc3FGM,XXArc3FGA,XXArc3Frequency,XXArc3Accuracy,XXUnblockedArc3Accuracy,XXArc3PctAssisted,XXArc3PctBlocked,XXNonHeaveArc3FGM,XXNonHeaveArc3FGA,XXNonHeaveArc3Accuracy,XXHeaveAttempts,XXHeaveMakes,XXAssists,XXAssistPoints,XXTwoPtAssists,XXThreePtAssists,XXAtRimAssists,XXShortMidRangeAssists,XXLongMidRangeAssists,XXCorner3Assists,XXArc3Assists,XXRebounds,XXDefRebounds,XXFTDefRebounds,XXDefFTReboundPct,XXDefTwoPtRebounds,XXDefTwoPtReboundPct,XXDefThreePtRebounds,XXDefThreePtReboundPct,XXDefFGReboundPct,XXOffRebounds,XXFTOffRebounds,XXOffFTReboundPct,XXOffTwoPtRebounds,XXOffTwoPtReboundPct,XXOffThreePtRebounds,XXOffThreePtReboundPct,XXOffFGReboundPct,XXDefAtRimReboundPct,XXDefShortMidRangeReboundPct,XXDefLongMidRangeReboundPct,XXDefArc3ReboundPct,XXDefCorner3ReboundPct,XXOffAtRimReboundPct,XXOffShortMidRangeReboundPct,XXOffLongMidRangeReboundPct,XXOffArc3ReboundPct,XXOffCorner3ReboundPct,XXSelfOReb,XXSelfORebPct,XXTurnovers,XXLiveBallTurnovers,XXDeadBallTurnove

In [55]:
def create_sim(df, start):
    sim_df = df.iloc[:start, :][
    ["GameID", "Elo", "XXElo", "Temp", "XXTemp", "Team", "XXTeam", "Points", "XXPoints", "Wins", "XXWins",
     "OffRank", "DefRank", "XXOffRank", "XXDefRank"]]
    count = len(sim_df)
    while (count != len(df)):
        row = df.iloc[count, :]
        recent_home = get_recent(sim_df, row["Team"])
        recent_away = get_recent(sim_df, row["XXTeam"])
        home_elo = new_elo(recent_home)[row["Team"]]
        away_elo = new_elo(recent_away)[row["XXTeam"]]
        home_points = home_svr.predict(row[
            ["Elo", "XXElo", "Points", "XXPoints", "Wins", "XXWins", "OffRank",
             "DefRank", "XXOffRank", "XXDefRank"]].values.reshape(1, -1))[0]
        away_points = away_svr.predict(row[
            ["Elo", "XXElo", "Points", "XXPoints", "Wins", "XXWins", "OffRank",
             "DefRank", "XXOffRank", "XXDefRank"]].values.reshape(1, -1))[0]
        home_gp = get_gp(data[idx], df.iloc[count]["Team"], df.iloc[count]["GameID"])
        away_gp = get_gp(data[idx], df.iloc[count]["XXTeam"], df.iloc[count]["GameID"])
        home_ppg = compute_avg(recent_home, df.iloc[count]["Team"], home_gp)
        away_ppg = compute_avg(recent_away, df.iloc[count]["XXTeam"], away_gp)
        home_wins = get_wins(recent_home, df.iloc[count]["Team"])
        away_wins = get_wins(recent_away, df.iloc[count]["XXTeam"])
        home_off_rank = stat_ranking(sim_df, count - 1, "Points", "off", "Team")
        home_def_rank = stat_ranking(sim_df, count - 1, "XXPoints", "def", "Team")
        away_off_rank = stat_ranking(sim_df, count - 1, "Points", "off", "XXTeam")
        away_def_rank = stat_ranking(sim_df, count - 1, "XXPoints", "def", "XXTeam")
        new_row = {"GameID": row["GameID"], "Team": row["Team"], "Elo": home_elo, "Temp": home_points,
                   "Points": home_ppg, "XXTeam": row["XXTeam"], "XXElo": away_elo, "XXTemp": away_points,
                   "XXPoints": away_ppg, "Wins": home_wins, "XXWins": away_wins, "OffRank": home_off_rank,
                   "DefRank": home_def_rank, "XXOffRank": away_off_rank, "XXDefRank": away_def_rank}
        new_row = pd.Series(new_row)
        sim_df.loc[count] = new_row
        count += 1
        
    return sim_df

In [116]:
data = pd.read_csv(str(Path.cwd()) + '/final_updated.csv')
seasons = data.Season.unique()
data = [data.loc[data.Season == season] for season in seasons]
for idx, df in enumerate(data):
    data[idx] = df.drop(['Season'], axis=1)
    
X = [df.loc[df['OffPoss'] != 0] for df in data]
for idx, df in enumerate(X):
    X[idx] = df.dropna(axis=0)
    
home_y1 = [df.Temp for df in X]
y2 = [df.W_L for df in X]
away_y1 = [df.XXTemp for df in X]
games = [df.GameID for df in X]
for idx, df in enumerate(X):
    X[idx] = df[["GameID", "Elo", "XXElo", "Temp", "XXTemp", "Team", "XXTeam", "Points", "XXPoints",
                 "OffRank", "DefRank", "XXOffRank", "XXDefRank"]]
    X[idx] = df.reset_index(drop=True)
    home_y1[idx] = home_y1[idx].reset_index(drop=True)
    y2[idx] = y2[idx].reset_index(drop=True)
    away_y1[idx] = away_y1[idx].reset_index(drop=True)

In [132]:
results = {}
sim_count = 2
for sim in tqdm(range(sim_count)):
    results[sim] = {}
    for idx, df in enumerate(X):
        results[sim][seasons[idx]] = {}
        # clf = svm.SVC(kernel="rbf", C=46, random_state=sim)
        home_svr = SVR(kernel="rbf", C=700)
        away_svr = SVR(kernel="rbf", C=2**15)
        X_data = pd.concat([X[i] for i in range(len(X)) if i != idx], axis=0)
        X_data = X_data[["Elo", "XXElo", "Points", "XXPoints", "Wins", "XXWins",
                         "OffRank", "DefRank", "XXOffRank", "XXDefRank"]]
        # y2_data = pd.concat([y2[i] for i in range(len(y2)) if i != idx], axis=0)
        home_y1_data = pd.concat([home_y1[i] for i in range(len(home_y1)) if i != idx], axis=0)
        away_y1_data = pd.concat([away_y1[i] for i in range(len(away_y1)) if i != idx], axis=0)
        # clf.fit(X_data, y2_data)
        X_train, X_test, home_y_train, home_y_test = train_test_split(
            X_data, home_y1_data, test_size=0.15, random_state=sim, shuffle=True)
        X_train, X_test, away_y_train, away_y_test = train_test_split(
            X_data, away_y1_data, test_size=0.15, random_state=sim, shuffle=True)
        home_svr.fit(X_train.values, home_y_train.values)
        away_svr.fit(X_train.values, away_y_train.values)
        # Simulate remaining games from each game
        game_num = find_start(df)
        while (game_num != len(df)):
            sim_df = create_sim(df, game_num)
            results[sim][seasons[idx]][df.iloc[game_num]["GameID"]] = sim_df
            game_num += 1

  0%|                                                   | 0/2 [9:58:55<?, ?it/s]


KeyboardInterrupt: 

In [135]:
results[0]['13-14']

{'2013-11-26ATLvORL':                  GameID   Elo  XXElo        Temp      XXTemp Team XXTeam  \
 0     2013-11-04HOUvLAC  1603   1578  137.000000  118.000000  LAC    HOU   
 1     2013-11-06BOSvUTA  1448   1492   97.000000   87.000000  BOS    UTA   
 2     2013-11-07HOUvLAL  1574   1522   98.000000   99.000000  HOU    LAL   
 3     2013-11-08CHIvUTA  1492   1481   97.000000   73.000000  CHI    UTA   
 4     2013-11-08CLEvPHI  1446   1365   94.000000   79.000000  PHI    CLE   
 ...                 ...   ...    ...         ...         ...  ...    ...   
 1115  2014-04-16LALvSAS  1743   1388  116.201945  101.302033  SAS    LAL   
 1116  2014-04-16MIAvPHI  1746   1203  114.420493   96.904140  MIA    PHI   
 1117  2014-04-16MINvUTA  1575   1329  108.803672   96.555006  MIN    UTA   
 1118  2014-04-16NYKvTOR  1467   1612  100.168087   99.832835  NYK    TOR   
 1119  2014-04-16PHXvSAC  1332   1539  100.521155  103.622660  SAC    PHO   
 
           Points    XXPoints  Wins  XXWins  OffRank 

In [127]:
test1 = results[0]["13-14"]['2013-11-26ATLvORL']

In [128]:
test2 = results[0]["13-14"]['2013-11-26ATLvORL']

In [131]:
calc_records(sim_df)

{'West': [('OKC', 69),
  ('SAS', 69),
  ('LAC', 64),
  ('HOU', 58),
  ('POR', 57),
  ('GSW', 57),
  ('DAL', 53),
  ('MEM', 49),
  ('MIN', 47),
  ('PHO', 46),
  ('DEN', 39),
  ('LAL', 28),
  ('NOP', 24),
  ('UTA', 17),
  ('SAC', 17)],
 'East': [('MIA', 66),
  ('IND', 65),
  ('CHI', 53),
  ('TOR', 50),
  ('WAS', 46),
  ('ATL', 42),
  ('BRK', 39),
  ('NYK', 38),
  ('CHA', 33),
  ('BOS', 28),
  ('DET', 24),
  ('CLE', 21),
  ('ORL', 13),
  ('PHI', 10),
  ('MIL', 8)],
 'NBA': [('OKC', 69),
  ('SAS', 69),
  ('MIA', 66),
  ('IND', 65),
  ('LAC', 64),
  ('HOU', 58),
  ('POR', 57),
  ('GSW', 57),
  ('CHI', 53),
  ('DAL', 53),
  ('TOR', 50),
  ('MEM', 49),
  ('MIN', 47),
  ('PHO', 46),
  ('WAS', 46),
  ('ATL', 42),
  ('DEN', 39),
  ('BRK', 39),
  ('NYK', 38),
  ('CHA', 33),
  ('BOS', 28),
  ('LAL', 28),
  ('DET', 24),
  ('NOP', 24),
  ('CLE', 21),
  ('UTA', 17),
  ('SAC', 17),
  ('ORL', 13),
  ('PHI', 10),
  ('MIL', 8)]}

In [59]:
def calc_records(sim_df):
    results = {}
    for team in sim_df.Team.unique():
        wins = 0
        team_df = sim_df[(sim_df["Team"] == team) | (sim_df["XXTeam"] == team)]
        final = team_df.iloc[-1]
        if final.Team == team:
            wins = final.Wins
            if final.Temp > final.XXTemp:
                wins += 1
        elif final.XXTeam == team:
            wins = final.XXWins
            if final.XXTemp > final.Temp:
                wins += 1
                
        results[team] = wins
        
    WEST = ["LAL", "DAL", "HOU", "SAC", "GSW", "NOP", "SAS", "MIN", "UTA", "PHO", "LAC", "DEN",
            "MEM", "POR", "OKC"]
    EAST = [team for team in data[0]["Team"] if team not in WEST]
    west_records = sorted([(k, itm) for k, itm in results.items() if k in WEST], key=lambda x: x[1], reverse=True)
    east_records = sorted([(k, itm) for k, itm in results.items() if k in EAST], key=lambda x: x[1], reverse=True)
    nba_records = sorted([(k, itm) for k, itm in results.items()], key=lambda x: x[1], reverse=True)
        
    return {"West": west_records, "East": east_records, "NBA": nba_records}